In [2]:
import dill as pickle
from collections import Counter
from fuzzywuzzy import fuzz
import itertools
from IPython.display import clear_output
import time
import numpy as np
import math

In [2]:
with open("files/my_mwe_files/top_candidates_from_papers.pkl", "rb") as f:
    top_candidates_from_papers = pickle.load(f)
overall_top_mwe_expression_candidates = Counter()
for k,v in top_candidates_from_papers.items():
    for k_,v_ in v.items():
        overall_top_mwe_expression_candidates[k_] += v_

overall_top_mwe_expression_candidates

Counter({('somatic', 'alterations'): 34.0,
         ('colorectal', 'carcinoma'): 288.5,
         ('low-depth-of-coverage', 'whole-genome'): 8.0,
         ('hypermutated', 'three-quarters'): 8.5,
         ('the', 'expected'): 9.0,
         ('somatic', 'mismatch-repair'): 8.5,
         ('POLE', 'mutations'): 8.5,
         ('Recurrent', 'copy-number'): 8.5,
         ('copy-number', 'alterations'): 8.0,
         ('alterations', 'include'): 8.0,
         ('drug-targetable', 'amplifications'): 8.0,
         ('Recurrent', 'copy-number', 'alterations'): 8.333333333333334,
         ('copy-number', 'alterations', 'include'): 8.0,
         ('of', 'gene'): 54.0,
         ('gene', 'expression'): 2247.5,
         ('of', 'gene', 'expression'): 61.99999999999999,
         ('normalization', 'strategy'): 9.0,
         ('constitutively', 'expressed'): 34.0,
         ('control', 'gene'): 13.0,
         ('robust', 'strategy'): 9.0,
         ('strategy', 'to'): 9.0,
         ('expressed', 'genes'): 79.0,
  

In [3]:
all_mwe = list(overall_top_mwe_expression_candidates.keys())
all_mwe_as_strings = [" ".join(mwe_tup) for mwe_tup in all_mwe ]

In [4]:
len(all_mwe_as_strings)

95655

In [7]:
pairwise_comparison = {}
each_run = []
for i in range(len(all_mwe_as_strings)):
    start_time = time.time()
    one_v_rest_ratio = {}
    one_v_rest_partial_ratio = []
    for j in range(i, len(all_mwe_as_strings)):
        combo = (all_mwe_as_strings[i], all_mwe_as_strings[j])
        one_v_rest_ratio[combo] = fuzz.ratio(combo[0],combo[1])
        if fuzz.partial_ratio(combo[0],combo[1]) == 100:
            one_v_rest_partial_ratio.append(combo[1])
    ratio_threshold = np.mean(list(one_v_rest_ratio.values())) + 4*np.std(list(one_v_rest_ratio.values()))
    similar_mwes = {}
    for k, v in one_v_rest_ratio.items():
        #print(v)
        if v > ratio_threshold and k[1] not in one_v_rest_partial_ratio:
            similar_mwes[k[1]] = v
    pairwise_comparison[all_mwe_as_strings[i]] = {"similar_mwes" : similar_mwes, 'sub_mwes' : one_v_rest_partial_ratio}
    end_time = time.time()
    each_run.append((end_time - start_time) / (len(all_mwe_as_strings) - i))
    #clear_output(wait=False)
    #print((np.mean(each_run) *  sum(list(range(2* (len(all_mwe_as_strings) + 1 - i)))))/ 60)
    

In [10]:
with open("pairwise_comparisons_mwe.pkl" , "wb") as f:
    pickle.dump(pairwise_comparison,f)

In [3]:
with open("pairwise_comparisons_mwe.pkl" , "rb") as f:
    pairwise_comparison = pickle.load(f)

In [6]:
pairwise_comparison['somatic alterations']['similar_mwes']

{'epigenetic alterations': 73,
 'genetic alterations': 79,
 'somatic mutation': 80,
 'somatic mutations': 83,
 'somatic insertion': 78,
 'Genetic alterations': 79,
 'somatic mutations in': 77,
 'somatic BAX mutations': 75,
 'biochemical alterations': 76,
 'genomic alterations': 84,
 'of genetic alterations': 78,
 'methylation alterations': 76,
 'and alteration': 73,
 'somatic muations': 80,
 'Epigenetic alterations': 73,
 'somatic genetic alterations': 83,
 'somatic hypermutation': 75,
 'two alterations': 76,
 'somatic β-catenin mutations': 74,
 'Somatic mutations': 78,
 'metabolic alterations': 80,
 'somatic inactivation': 77,
 'p53-somatic mutations': 75,
 'genetic alterations in': 73}